<a href="https://colab.research.google.com/github/MatteusMoroni/CTPatrickRamos/blob/main/AI_Finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install gradio
%pip -q install google-genai
!pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.

In [2]:
# -*- coding: utf-8 -*-
"""AI Finance

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/14JEvfLqRUEu_G4gem4Klu39yRwuaJP5i
"""

# Commented out IPython magic to ensure Python compatibility.
!pip -q install gradio
# %pip -q install google-genai
!pip -q install google-adk

import os
from google.colab import userdata
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
import gradio as gr

warnings.filterwarnings("ignore")

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')



In [15]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

def agente_orcamento(entradas, despesas):
    gerador_orcamento = Agent(
        name="agente_orcamento",
        model="gemini-2.0-flash",
        instruction="""
                    Você é um analista financeiro experiente especializado em orçamentos familiares brasileiros.
                    Sua tarefa é analisar a entrada e despesa familiar fornecidas e criar um orçamento mensal detalhado e realista.

                    O orçamento deve ser dividido nos seguintes itens (com porcentagens e valores):
                    - Custo Fixo: Valores essenciais para a manutenção do lar (alimentação, aluguel, transporte, etc.).
                    - Conforto: Despesas não essenciais, mas que melhoram a qualidade de vida (restaurantes, streaming, hobbies, etc.).
                    - Metas: Valor reservado para objetivos financeiros de curto e médio prazo (viagens, eletrônicos, etc.).
                    - Conhecimento: Investimento em educação e desenvolvimento pessoal (livros, cursos, etc.).
                    - Investimentos: Recursos destinados ao futuro financeiro (aposentadoria, independência financeira, etc.).

                    **Instruções Adicionais:**
                    - Apresente o orçamento de forma clara e organizada, em uma lista.
                    - Inclua porcentagens e valores em reais (R$).
                    - Use abreviações comuns para porcentagem (%) e real (R$).
                    - Seja conciso e direto. Não inclua saudações ou explicações extensas.
                    - se possível destine até 20% em investimento, sem sacrificar os outros orçamentos
                    - Exemplo de formato de saída esperado:
                      - Custo Fixo: 50% - R$ 2000,00
                      - Conforto: 20% - R$ 800,00
        """,
        description="Agente responsável por criar um orçamento",
        tools=[google_search]
    )
    entrada_do_agente_orcamento = f"Entradas mensais: {entradas}\nDespesas fixas mensais: {despesas}"
    orcamento = call_agent(gerador_orcamento, entrada_do_agente_orcamento)
    return orcamento

def agente_investimento(orcamento, objetivo):
    gerador_plano_investimento = Agent(
        name="agente_investimento",
        model="gemini-2.0-flash",
        instruction="""
          Você é um especialista em investimentos financeiros com foco no mercado brasileiro.
          Sua tarefa é analisar o orçamento familiar e o objetivo de investimento fornecidos para criar um plano de investimento mensal personalizado e adequado para pessoas leigas.

          **Instruções:**
          1.  Analise o orçamento fornecido para determinar a quantia disponível para investimento.
          2.  Utilize a ferramenta de busca do Google (google_search) para identificar os melhores investimentos (renda fixa, renda variável, ETFs, etc.)
              considerando o contexto brasileiro, as tendências atuais do mercado, a opinião de especialistas e o histórico de desempenho dos ativos.
          3.  Crie um plano de investimento mensal claro e conciso, especificando:
              -   As opções de investimento recomendadas.
              - 3 atimos recomendações para cada investimento, busque com a ferramente de busca do Google (google_search).
              -   A porcentagem do valor disponível a ser alocado em cada investimento.
              -   Justificativas para as recomendações, explicando os riscos e potenciais retornos de cada opção.
          4.  Apresente o plano de investimento de forma organizada e fácil de entender, utilizando uma tabela para resumir as informações.
          5.  Não inclua avisos genéricos sobre riscos de investimento. Foque em fornecer informações objetivas e práticas.


          **Formato de Saída Esperado:**

          Plano de Investimento:
          [Texto explicativo do plano]

          Tabela de Investimentos:
          | Investimento    | Valor (R$) | Porcentagem |
          |----------------|------------|-------------|
          | [Nome do Inv.] | [Valor]     | [Porcentagem] |
          | ...             | ...         | ...         |

    """,
        description="Agente responsável por criar um plano de investimento",
        tools=[google_search]
    )
    entrada_do_agente_investimento = f"Orçamento mensal: {orcamento}\n objetivos de investimento: {objetivo}"
    plano_investimento = call_agent(gerador_plano_investimento, entrada_do_agente_investimento)
    return plano_investimento

def analyze_finance(renda_mensal, despesas_fixas, objetivo_investimento):
    orcamento = agente_orcamento(renda_mensal, despesas_fixas)
    plano = agente_investimento(orcamento, objetivo_investimento)
    return orcamento,  plano

iface = gr.Interface(
    fn=analyze_finance,
    inputs=[
        gr.Textbox(label="Informe seus ganhos mensais (R$)"),
        gr.Textbox(label="Informe suas despesas fixas mensais (R$)"),
        gr.Textbox(label="Qual seu objetivo financeiro para investimento?"),
    ],
    outputs=[
        gr.Markdown(label="Orçamento"),
        gr.Markdown(label="Plano de Investimento")
    ],
    title="Análise Financeira com IA",
    description="Forneça seus ganhos, despesas e objetivo de investimento para obter um orçamento e plano de investimento personalizados."
)

iface.launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://162b23594a92e53c42.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://162b23594a92e53c42.gradio.live
